In [ ]:
import sys
import os
import logging
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pytz

import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
from ta.momentum import RSIIndicator

# ============================
# Logging Configuration
# ============================

# Create a custom logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)  # Set to DEBUG to capture all levels of logs

# Create handlers
file_handler = logging.FileHandler('backtest.log')
file_handler.setLevel(logging.INFO)  # File handler captures INFO and above

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)  # Console handler captures INFO and above

# Create formatters and add them to handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers to the logger
if not logger.handlers:
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

# ============================
# Utility Functions
# ============================

def init_mt5_connection(login, password, server):
    """
    Initialize connection to MetaTrader 5.

    Parameters:
        login (int): MT5 account login number.
        password (str): MT5 account password.
        server (str): MT5 server name.

    Exits the program if connection fails.
    """
    print("Initializing connection to MetaTrader 5...")
    logger.info("Attempting to connect to MetaTrader 5.")
    if not mt5.initialize(login=login, password=password, server=server):
        error_msg = f"initialize() failed, error code = {mt5.last_error()}"
        logger.error(error_msg)
        print(f"Error: {error_msg}")
        sys.exit("MT5 initialization failed. Check 'backtest.log' for details.")
    logger.info("Connected to MetaTrader 5 successfully.")
    print("Successfully connected to MetaTrader 5.")

def fetch_ohlc_data(symbol, timeframe, start_date, end_date):
    """
    Fetch OHLC data from MT5.

    Parameters:
        symbol (str): Symbol to fetch data for.
        timeframe (int): MT5 timeframe constant (e.g., mt5.TIMEFRAME_H1).
        start_date (datetime): Start date for data.
        end_date (datetime): End date for data.

    Returns:
        pd.DataFrame: DataFrame containing OHLC data.
    """
    print(f"Fetching OHLC data for {symbol} from {start_date} to {end_date}...")
    logger.info(f"Fetching OHLC data for {symbol} from {start_date} to {end_date}.")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    if data is None or len(data) == 0:
        error_msg = f"Failed to fetch data for {symbol}."
        logger.error(error_msg)
        print(f"Error: {error_msg}")
        return None
    df = pd.DataFrame(data)
    # Make 'time' timezone-aware (UTC)
    df['time'] = pd.to_datetime(df['time'], unit='s').dt.tz_localize('UTC')
    df.rename(
        columns={
            'open': 'Open',
            'high': 'High',
            'low': 'Low',
            'close': 'Close',
            'tick_volume': 'Volume'
        },
        inplace=True
    )
    logger.info(f"Successfully fetched {len(df)} data points for {symbol}.")
    print(f"Successfully fetched {len(df)} data points for {symbol}.")
    return df[['time', 'Open', 'High', 'Low', 'Close', 'Volume']]

def add_technical_indicators(ohlc_data, ema_short_span=10, ema_long_span=30, macd_short_span=12, macd_long_span=26, macd_signal_span=9):
    """
    Add technical indicators to OHLC data.

    Parameters:
        ohlc_data (pd.DataFrame): DataFrame with OHLC data.
        ema_short_span (int): Span for short EMA.
        ema_long_span (int): Span for long EMA.
        macd_short_span (int): Short span for MACD.
        macd_long_span (int): Long span for MACD.
        macd_signal_span (int): Signal span for MACD.

    Returns:
        pd.DataFrame: DataFrame with added indicators.
    """
    logger.info("Adding technical indicators to OHLC data.")
    ohlc_data['rsi'] = RSIIndicator(ohlc_data['Close'], window=14).rsi()
    ohlc_data['short_ma'] = ohlc_data['Close'].ewm(span=ema_short_span, adjust=False).mean()
    ohlc_data['long_ma'] = ohlc_data['Close'].ewm(span=ema_long_span, adjust=False).mean()
    short_ema = ohlc_data['Close'].ewm(span=macd_short_span, adjust=False).mean()
    long_ema = ohlc_data['Close'].ewm(span=macd_long_span, adjust=False).mean()
    ohlc_data['macd'] = short_ema - long_ema
    ohlc_data['macd_signal'] = ohlc_data['macd'].ewm(span=macd_signal_span, adjust=False).mean()
    logger.info("Technical indicators added successfully.")
    return ohlc_data

def load_and_align_data(ohlc_data, prediction_file):
    """
    Load prediction data and align it with OHLC data.

    Parameters:
        ohlc_data (pd.DataFrame): DataFrame with OHLC and indicators.
        prediction_file (str): Path to CSV file containing predictions.

    Returns:
        pd.DataFrame: Merged DataFrame with predictions.
    """
    print(f"Loading prediction data from '{prediction_file}'...")
    logger.info(f"Loading prediction data from '{prediction_file}'.")
    if not os.path.exists(prediction_file):
        error_msg = f"Prediction file '{prediction_file}' does not exist."
        logger.error(error_msg)
        print(f"Error: {error_msg} Skipping this prediction file.")
        return None
    try:
        pred_df = pd.read_csv(prediction_file, parse_dates=['time'])
        if 'prediction' not in pred_df.columns:
            error_msg = f"'prediction' column not found in {prediction_file}."
            logger.error(error_msg)
            print(f"Error: {error_msg} Skipping this prediction file.")
            return None
        # Make 'time' timezone-aware (UTC)
        pred_df['time'] = pd.to_datetime(pred_df['time']).dt.tz_localize('UTC')
    except Exception as e:
        error_msg = f"Error loading prediction file '{prediction_file}': {e}"
        logger.error(error_msg)
        print(f"Error: {error_msg} Skipping this prediction file.")
        return None

    # Verify dtypes
    print(f"OHLC 'time' dtype: {ohlc_data['time'].dtype}")
    print(f"Prediction 'time' dtype: {pred_df['time'].dtype}")
    logger.debug(f"OHLC 'time' dtype: {ohlc_data['time'].dtype}")
    logger.debug(f"Prediction 'time' dtype: {pred_df['time'].dtype}")

    # Merge on 'time'
    try:
        merged = pd.merge(
            ohlc_data,
            pred_df[['time', 'prediction']],
            on='time',
            how='left'
        )
        # Shift predictions upwards by one to align with the next candle
        merged['prediction'] = merged['prediction'].shift(-1).fillna(0)
        logger.info(f"Merged prediction data from '{prediction_file}' with OHLC data.")
    except Exception as e:
        error_msg = f"Error merging data: {e}"
        logger.error(error_msg)
        print(f"Error: {error_msg}")
        return None

    # Validate merge
    if merged['prediction'].isnull().any():
        warning_msg = "Some 'prediction' values are NaN after merge."
        logger.warning(warning_msg)
        print(f"Warning: {warning_msg}")
    else:
        logger.info("All 'prediction' values successfully merged.")
        print("All 'prediction' values successfully merged.")

    print(f"Successfully loaded and aligned prediction data for '{prediction_file}'.")
    return merged

def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, trade_type):
    """
    Calculate Stop Loss (SL) and Take Profit (TP) prices.

    Parameters:
        entry_price (float): Entry price of the trade.
        risk_reward_ratio (str): Risk to reward ratio (e.g., "1:1").
        mean_candle_size (float): Average candle size for risk calculation.
        trade_type (str): "Buy" or "Sell".

    Returns:
        tuple: (sl_price, tp_price)
    """
    try:
        risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    except ValueError:
        error_msg = f"Invalid risk_reward_ratio format: {risk_reward_ratio}"
        logger.error(error_msg)
        print(f"Error: {error_msg}. Setting SL and TP to entry price.")
        return entry_price, entry_price  # No SL/TP set

    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part

    if trade_type == "Buy":
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    else:  # "Sell"
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount

    logger.debug(f"Calculated prices for {trade_type}: SL={sl_price}, TP={tp_price}")
    return sl_price, tp_price

def calculate_position_size(account_balance, risk_per_trade, entry_price, sl_price):
    """
    Calculate position size based on risk management.

    Parameters:
        account_balance (float): Current account balance.
        risk_per_trade (float): Fraction of account balance to risk per trade (e.g., 0.02 for 2%).
        entry_price (float): Entry price of the trade.
        sl_price (float): Stop loss price.

    Returns:
        int: Position size (number of units/contracts).
    """
    risk_amount = account_balance * risk_per_trade
    stop_loss_distance = abs(entry_price - sl_price)
    if stop_loss_distance == 0:
        warning_msg = "Stop loss distance is zero. Position size set to 0."
        logger.warning(warning_msg)
        print(f"Warning: {warning_msg}")
        return 0  # Avoid division by zero
    position_size = risk_amount / stop_loss_distance
    # Round to the nearest whole number
    position_size = int(round(position_size))
    logger.debug(f"Calculated position size: {position_size} units.")
    return position_size

# ============================
# Strategy Classes
# ============================

class FilterStrategy(Strategy):
    """
    Strategy with technical indicators filters and position sizing.
    """
    # Strategy parameters (to be set dynamically)
    pair_name = None
    signal_type = None
    risk_per_trade = 0.02  # 2% of account balance
    rsi_buy = 60
    rsi_sell = 40
    risk_reward_ratio = "1:1"        # e.g., "1:2"
    mean_candle_size = 0.005         # e.g., 0.005
    volume_threshold = 1000
    ema_short_span = 10
    ema_long_span = 30
    macd_short_span = 12
    macd_long_span = 26
    macd_signal_span = 9

    def init(self):
        # Indicators are already added to the data
        pass

    def next(self):
        entry_price = self.data.Close[-1]
        buy_cond = (
            (self.data.rsi[-1] < self.rsi_buy) &
            (self.data.short_ma[-1] > self.data.long_ma[-1]) &
            (self.data.macd[-1] > self.data.macd_signal[-1]) &
            (self.data.Volume[-1] > self.volume_threshold)
        )
        sell_cond = (
            (self.data.rsi[-1] > self.rsi_sell) &
            (self.data.short_ma[-1] < self.data.long_ma[-1]) &
            (self.data.macd[-1] < self.data.macd_signal[-1]) &
            (self.data.Volume[-1] > self.volume_threshold)
        )

        if self.signal_type == 'Buy':
            if self.data.prediction[-1] == 1 and buy_cond:
                sl_price, tp_price = calculate_prices(
                    entry_price, self.risk_reward_ratio, 
                    self.mean_candle_size, "Buy"
                )
                position_size = calculate_position_size(
                    self.equity, self.risk_per_trade, entry_price, sl_price
                )
                if position_size > 0:
                    self.buy(size=position_size, sl=sl_price, tp=tp_price)
                    logger.info(f"Executed Buy: Entry={entry_price}, SL={sl_price}, TP={tp_price}, Size={position_size}")
                    print(f"Executed Buy: Entry={entry_price}, SL={sl_price}, TP={tp_price}, Size={position_size}")
        else:  # 'Sell'
            if self.data.prediction[-1] == 1 and sell_cond:
                sl_price, tp_price = calculate_prices(
                    entry_price, self.risk_reward_ratio, 
                    self.mean_candle_size, "Sell"
                )
                position_size = calculate_position_size(
                    self.equity, self.risk_per_trade, entry_price, sl_price
                )
                if position_size > 0:
                    self.sell(size=position_size, sl=sl_price, tp=tp_price)
                    logger.info(f"Executed Sell: Entry={entry_price}, SL={sl_price}, TP={tp_price}, Size={position_size}")
                    print(f"Executed Sell: Entry={entry_price}, SL={sl_price}, TP={tp_price}, Size={position_size}")

class WithoutFilterStrategy(Strategy):
    """
    Strategy without technical indicators filters and with position sizing.
    """
    # Strategy parameters (to be set dynamically)
    pair_name = None
    signal_type = None
    risk_per_trade = 0.02  # 2% of account balance
    risk_reward_ratio = "1:1"
    mean_candle_size = 0.005

    def init(self):
        # Indicators are already added to the data
        pass

    def next(self):
        entry_price = self.data.Close[-1]
        if self.signal_type == 'Buy':
            if self.data.prediction[-1] == 1:
                sl_price, tp_price = calculate_prices(
                    entry_price, self.risk_reward_ratio, 
                    self.mean_candle_size, "Buy"
                )
                position_size = calculate_position_size(
                    self.equity, self.risk_per_trade, entry_price, sl_price
                )
                if position_size > 0:
                    self.buy(size=position_size, sl=sl_price, tp=tp_price)
                    logger.info(f"Executed Buy: Entry={entry_price}, SL={sl_price}, TP={tp_price}, Size={position_size}")
                    print(f"Executed Buy: Entry={entry_price}, SL={sl_price}, TP={tp_price}, Size={position_size}")
        else:  # 'Sell'
            if self.data.prediction[-1] == 1:
                sl_price, tp_price = calculate_prices(
                    entry_price, self.risk_reward_ratio, 
                    self.mean_candle_size, "Sell"
                )
                position_size = calculate_position_size(
                    self.equity, self.risk_per_trade, entry_price, sl_price
                )
                if position_size > 0:
                    self.sell(size=position_size, sl=sl_price, tp=tp_price)
                    logger.info(f"Executed Sell: Entry={entry_price}, SL={sl_price}, TP={tp_price}, Size={position_size}")
                    print(f"Executed Sell: Entry={entry_price}, SL={sl_price}, TP={tp_price}, Size={position_size}")

# ============================
# Backtesting Functions
# ============================

def run_filter_variations(
    ohlc_data, pair_name, signal_type, margin, risk_reward_ratio, 
    mean_candle_size, parameter_combos, results_summary
):
    """
    Run backtests for multiple parameter combinations using FilterStrategy.

    Parameters:
        ohlc_data (pd.DataFrame): OHLC data with indicators and predictions.
        pair_name (str): Name of the trading pair.
        signal_type (str): 'Buy' or 'Sell'.
        margin (float): Margin requirement for the trade.
        risk_reward_ratio (str): Risk to reward ratio (e.g., "1:1").
        mean_candle_size (float): Average candle size for risk calculation.
        parameter_combos (list): List of parameter dictionaries.
        results_summary (list): List to append the results.
    """
    print(f"\nRunning FilterStrategy backtests for {pair_name} - {signal_type}...")
    logger.info(f"Running FilterStrategy backtests for {pair_name} - {signal_type}.")
    for idx, params in enumerate(parameter_combos, start=1):
        print(f"  Backtest {idx}/{len(parameter_combos)} with parameters: {params}")
        logger.info(f"Backtest {idx}/{len(parameter_combos)} for {pair_name} - {signal_type} with parameters: {params}")

        # Unpack parameters
        rsi_buy, rsi_sell = params['rsi']
        ema_short_span = params['ema_short_span']
        ema_long_span = params['ema_long_span']
        macd_short_span = params['macd_short_span']
        macd_long_span = params['macd_long_span']
        macd_signal_span = params['macd_signal_span']

        # Add technical indicators with current parameters
        df = add_technical_indicators(
            ohlc_data.copy(),
            ema_short_span=ema_short_span,
            ema_long_span=ema_long_span,
            macd_short_span=macd_short_span,
            macd_long_span=macd_long_span,
            macd_signal_span=macd_signal_span
        )

        # Update FilterStrategy parameters
        FilterStrategy.pair_name = pair_name
        FilterStrategy.signal_type = signal_type
        FilterStrategy.rsi_buy = rsi_buy
        FilterStrategy.rsi_sell = rsi_sell
        FilterStrategy.risk_reward_ratio = risk_reward_ratio
        FilterStrategy.mean_candle_size = mean_candle_size
        FilterStrategy.ema_short_span = ema_short_span
        FilterStrategy.ema_long_span = ema_long_span
        FilterStrategy.macd_short_span = macd_short_span
        FilterStrategy.macd_long_span = macd_long_span
        FilterStrategy.macd_signal_span = macd_signal_span

        # Initialize Backtest
        bt = Backtest(
            df.set_index('time'),
            FilterStrategy,
            cash=10000,  # Starting cash
            commission=0.0003,  # Commission per trade (e.g., 0.03%)
            margin=margin,
            exclusive_orders=True  # Whether to close existing orders before opening new ones
        )
        try:
            stats = bt.run()
            logger.info(f"Backtest {idx} completed for {pair_name} - {signal_type}. Final Equity: ${stats['Equity Final [$]']:.2f}, Return: {stats['Return [%]']:.2f}%")
            print(f"    Backtest {idx} completed. Final Equity: ${stats['Equity Final [$]']:.2f}, Return: {stats['Return [%]']:.2f}%")
        except Exception as e:
            error_msg = f"Backtest {idx} failed for {pair_name} - {signal_type} with params {params}: {e}"
            logger.error(error_msg)
            print(f"    Error: {error_msg}")
            continue

        # Access the equity curve
        eq_curve = stats['_equity_curve'].copy()

        # Prepare plot directory
        pair_folder = os.path.join("plots", pair_name)
        os.makedirs(pair_folder, exist_ok=True)

        # Plot Equity Curve
        plt.figure(figsize=(10, 6))
        plt.plot(eq_curve.index, eq_curve['Equity'], label=f"{pair_name} {signal_type} Filter")
        plt.title(
            f"{pair_name} - {signal_type} Filter\nRSI: {rsi_buy}/{rsi_sell}, "
            f"EMA: {ema_short_span}/{ema_long_span}, MACD: {macd_short_span}/{macd_long_span}/{macd_signal_span}, "
            f"RR: {risk_reward_ratio}"
        )
        plt.xlabel("Time")
        plt.ylabel("Equity")
        plt.legend()
        plt.grid(True)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
        plt.gcf().autofmt_xdate()  # Rotate date labels
        plot_filename = os.path.join(
            pair_folder,
            f"{pair_name}_{signal_type}_Filter_RSI_{rsi_buy}_{rsi_sell}_EMA_{ema_short_span}_{ema_long_span}_MACD_{macd_short_span}_{macd_long_span}_{macd_signal_span}_RR_{risk_reward_ratio.replace(':','_')}.png"
        )
        plt.savefig(plot_filename)
        plt.close()  # Use close to avoid displaying during batch runs
        logger.info(f"Equity curve plot saved to '{plot_filename}'.")
        print(f"    Equity curve plot saved to '{plot_filename}'.")

        # Append results to summary
        results_summary.append({
            "Pair": pair_name,
            "Signal": signal_type,
            "Approach": "Filter",
            "RSI (Buy/Sell)": f"{rsi_buy}/{rsi_sell}",
            "EMA (Short/Long)": f"{ema_short_span}/{ema_long_span}",
            "MACD (Short/Long/Signal)": f"{macd_short_span}/{macd_long_span}/{macd_signal_span}",
            "Risk/Reward": risk_reward_ratio,
            "Final Equity": f"${stats['Equity Final [$]']:.2f}",
            "Return (%)": f"{stats['Return [%]']:.2f}",
            "Max Drawdown (%)": f"{stats['Max. Drawdown [%]']:.2f}",
            "Profit Factor": f"{stats['Profit Factor']:.2f}" if 'Profit Factor' in stats else "N/A",
            "Sharpe Ratio": f"{stats['Sharpe Ratio']:.2f}" if 'Sharpe Ratio' in stats else "N/A",
            "Win Rate (%)": f"{stats['Win Rate [%]']:.2f}" if 'Win Rate [%]' in stats else "N/A",
            "Total Trades": stats['# Trades'] if '# Trades' in stats else "N/A",
            "Avg Trade Duration": stats['Avg. Trade Duration'] if 'Avg. Trade Duration' in stats else "N/A",
            "Equity Curve Image": plot_filename,
        })

def run_without_filter_once(
    ohlc_data, pair_name, signal_type, margin, 
    risk_reward_ratio, mean_candle_size, results_summary
):
    """
    Run backtest once using WithoutFilterStrategy.

    Parameters:
        ohlc_data (pd.DataFrame): OHLC data with indicators and predictions.
        pair_name (str): Name of the trading pair.
        signal_type (str): 'Buy' or 'Sell'.
        margin (float): Margin requirement for the trade.
        risk_reward_ratio (str): Risk to reward ratio (e.g., "1:1").
        mean_candle_size (float): Average candle size for risk calculation.
        results_summary (list): List to append the results.
    """
    print(f"\nRunning WithoutFilterStrategy backtest for {pair_name} - {signal_type}...")
    logger.info(f"Running WithoutFilterStrategy backtest for {pair_name} - {signal_type}.")

    # Update WithoutFilterStrategy parameters
    WithoutFilterStrategy.pair_name = pair_name
    WithoutFilterStrategy.signal_type = signal_type
    WithoutFilterStrategy.risk_reward_ratio = risk_reward_ratio
    WithoutFilterStrategy.mean_candle_size = mean_candle_size
    WithoutFilterStrategy.risk_per_trade = 0.02  # 2% of account balance

    # Initialize Backtest
    bt = Backtest(
        ohlc_data.set_index('time'),
        WithoutFilterStrategy,
        cash=10000,  # Starting cash
        commission=0.0003,  # Commission per trade (e.g., 0.03%)
        margin=margin,
        exclusive_orders=True  # Whether to close existing orders before opening new ones
    )
    try:
        stats = bt.run()
        logger.info(f"WithoutFilter backtest completed for {pair_name} - {signal_type}. Final Equity: ${stats['Equity Final [$]']:.2f}, Return: {stats['Return [%]']:.2f}%")
        print(f"  WithoutFilter Backtest completed. Final Equity: ${stats['Equity Final [$]']:.2f}, Return: {stats['Return [%]']:.2f}%")
    except Exception as e:
        error_msg = f"WithoutFilter backtest failed for {pair_name} - {signal_type}: {e}"
        logger.error(error_msg)
        print(f"  Error: {error_msg}")
        return

    # Access the equity curve
    eq_curve = stats['_equity_curve'].copy()

    # Prepare plot directory
    pair_folder = os.path.join("plots", pair_name)
    os.makedirs(pair_folder, exist_ok=True)

    # Plot Equity Curve
    plt.figure(figsize=(10, 6))
    plt.plot(eq_curve.index, eq_curve['Equity'], label=f"{pair_name} {signal_type} WithoutFilter", color='orange')
    plt.title(f"{pair_name} - {signal_type} WithoutFilter\nRR: {risk_reward_ratio}")
    plt.xlabel("Time")
    plt.ylabel("Equity")
    plt.legend()
    plt.grid(True)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gcf().autofmt_xdate()  # Rotate date labels
    plot_filename = os.path.join(
        pair_folder,
        f"{pair_name}_{signal_type}_WithoutFilter_RR_{risk_reward_ratio.replace(':','_')}.png"
    )
    plt.savefig(plot_filename)
    plt.close()  # Use close to avoid displaying during batch runs
    logger.info(f"Equity curve plot saved to '{plot_filename}'.")
    print(f"  Equity curve plot saved to '{plot_filename}'.")

    # Append results to summary
    results_summary.append({
        "Pair": pair_name,
        "Signal": signal_type,
        "Approach": "WithoutFilter",
        "RSI (Buy/Sell)": "N/A",
        "EMA (Short/Long)": "N/A",
        "MACD (Short/Long/Signal)": "N/A",
        "Risk/Reward": risk_reward_ratio,
        "Final Equity": f"${stats['Equity Final [$]']:.2f}",
        "Return (%)": f"{stats['Return [%]']:.2f}",
        "Max Drawdown (%)": f"{stats['Max. Drawdown [%]']:.2f}" if 'Max. Drawdown [%]' in stats else "N/A",
        "Profit Factor": f"{stats['Profit Factor']:.2f}" if 'Profit Factor' in stats else "N/A",
        "Sharpe Ratio": f"{stats['Sharpe Ratio']:.2f}" if 'Sharpe Ratio' in stats else "N/A",
        "Win Rate (%)": f"{stats['Win Rate [%]']:.2f}" if 'Win Rate [%]' in stats else "N/A",
        "Total Trades": stats['# Trades'] if '# Trades' in stats else "N/A",
        "Avg Trade Duration": stats['Avg. Trade Duration'] if 'Avg. Trade Duration' in stats else "N/A",
        "Equity Curve Image": plot_filename,
    })

def summarize_results(results_summary):
    """
    Summarize backtest results and save to CSV.

    Parameters:
        results_summary (list): List of result dictionaries.
    """
    if not results_summary:
        print("No results to summarize.")
        logger.warning("No results to summarize.")
        return
    df_summary = pd.DataFrame(results_summary)
    print("\nBacktest Summary:\n")
    print(df_summary.to_string(index=False))
    logger.info("Backtest Summary:")
    logger.info("\n" + df_summary.to_string(index=False))
    try:
        df_summary.to_csv("backtest_results_summary.csv", index=False)
        print("\nResults summary saved to 'backtest_results_summary.csv'.")
        logger.info("Results summary saved to 'backtest_results_summary.csv'.")
    except Exception as e:
        error_msg = f"Failed to save results summary: {e}"
        logger.error(error_msg)
        print(f"Error: {error_msg}")

# ============================
# Main Function
# ============================

def main():
    """
    Main function to orchestrate the backtesting process.
    """
    config = {
        'login': 51988090,
        'password': '1fMdV52$74EOcw',
        'server': 'ICMarketsEU-Demo',

        # Each pair has mean_candle_size, risk-reward ratio, and parameter ranges
        'EURUSD': {
            'symbol': 'EURUSD',
            'timeframe': mt5.TIMEFRAME_D1,
            'mean_candle_size': 0.009,
            'buy_prediction_file': 'pred_EURUSD_Buy copy.csv',
            'sell_prediction_file': 'pred_EURUSD_Sell.csv',
            'buy_risk_reward_ratio': "1:1",
            'sell_risk_reward_ratio': "1:1",
            'parameter_combos': [
                # Define each parameter set as a dictionary
                {'rsi': (60, 40), 'ema_short_span': 10, 'ema_long_span': 30, 'macd_short_span': 12, 'macd_long_span': 26, 'macd_signal_span': 9},
                {'rsi': (65, 40), 'ema_short_span': 12, 'ema_long_span': 28, 'macd_short_span': 13, 'macd_long_span': 25, 'macd_signal_span': 9},
                {'rsi': (70, 45), 'ema_short_span': 14, 'ema_long_span': 32, 'macd_short_span': 14, 'macd_long_span': 24, 'macd_signal_span': 10},
                {'rsi': (55, 35), 'ema_short_span': 8,  'ema_long_span': 25, 'macd_short_span': 11, 'macd_long_span': 24, 'macd_signal_span': 8},
                {'rsi': (60, 40), 'ema_short_span': 11, 'ema_long_span': 29, 'macd_short_span': 12, 'macd_long_span': 27, 'macd_signal_span': 10},
                {'rsi': (65, 45), 'ema_short_span': 13, 'ema_long_span': 31, 'macd_short_span': 14, 'macd_long_span': 28, 'macd_signal_span': 11},
                {'rsi': (70, 50), 'ema_short_span': 15, 'ema_long_span': 35, 'macd_short_span': 15, 'macd_long_span': 30, 'macd_signal_span': 12},
                {'rsi': (58, 38), 'ema_short_span': 9,  'ema_long_span': 26, 'macd_short_span': 12, 'macd_long_span': 25, 'macd_signal_span': 9},
                {'rsi': (63, 43), 'ema_short_span': 12, 'ema_long_span': 30, 'macd_short_span': 13, 'macd_long_span': 27, 'macd_signal_span': 10},
                {'rsi': (68, 48), 'ema_short_span': 14, 'ema_long_span': 34, 'macd_short_span': 14, 'macd_long_span': 28, 'macd_signal_span': 11},
                # Add more combinations as needed
            ],
            'margin': 0.05
        },
        'GBPUSD': {
            'symbol': 'GBPUSD',
            'timeframe': mt5.TIMEFRAME_D1,
            'mean_candle_size': 0.01,
            'buy_prediction_file': 'pred_GBPUSD_Buy copy.csv',
            'sell_prediction_file': 'pred_GBPUSD_Sell copy.csv',
            'buy_risk_reward_ratio': "1:1",
            'sell_risk_reward_ratio': "1:1",
            'parameter_combos': [
                {'rsi': (60, 40), 'ema_short_span': 10, 'ema_long_span': 30, 'macd_short_span': 12, 'macd_long_span': 26, 'macd_signal_span': 9},
                {'rsi': (65, 40), 'ema_short_span': 12, 'ema_long_span': 28, 'macd_short_span': 13, 'macd_long_span': 25, 'macd_signal_span': 9},
                {'rsi': (70, 45), 'ema_short_span': 14, 'ema_long_span': 32, 'macd_short_span': 14, 'macd_long_span': 24, 'macd_signal_span': 10},
                {'rsi': (55, 35), 'ema_short_span': 8,  'ema_long_span': 25, 'macd_short_span': 11, 'macd_long_span': 24, 'macd_signal_span': 8},
                {'rsi': (60, 40), 'ema_short_span': 11, 'ema_long_span': 29, 'macd_short_span': 12, 'macd_long_span': 27, 'macd_signal_span': 10},
                {'rsi': (65, 45), 'ema_short_span': 13, 'ema_long_span': 31, 'macd_short_span': 14, 'macd_long_span': 28, 'macd_signal_span': 11},
                {'rsi': (70, 50), 'ema_short_span': 15, 'ema_long_span': 35, 'macd_short_span': 15, 'macd_long_span': 30, 'macd_signal_span': 12},
                {'rsi': (58, 38), 'ema_short_span': 9,  'ema_long_span': 26, 'macd_short_span': 12, 'macd_long_span': 25, 'macd_signal_span': 9},
                {'rsi': (63, 43), 'ema_short_span': 12, 'ema_long_span': 30, 'macd_short_span': 13, 'macd_long_span': 27, 'macd_signal_span': 10},
                {'rsi': (68, 48), 'ema_short_span': 14, 'ema_long_span': 34, 'macd_short_span': 14, 'macd_long_span': 28, 'macd_signal_span': 11},
                # Add more combinations as needed
            ],
            'margin': 0.1
        },
        'AUDUSD': {
            'symbol': 'AUDUSD',
            'timeframe': mt5.TIMEFRAME_D1,
            'mean_candle_size': 0.008,
            'buy_prediction_file': 'pred_AUDUSD_Buy copy.csv',
            'sell_prediction_file': 'pred_AUDUSD_Sell copy.csv',
            'buy_risk_reward_ratio': "1:1",
            'sell_risk_reward_ratio': "1:1",
            'parameter_combos': [
                {'rsi': (60, 40), 'ema_short_span': 10, 'ema_long_span': 30, 'macd_short_span': 12, 'macd_long_span': 26, 'macd_signal_span': 9},
                {'rsi': (65, 40), 'ema_short_span': 12, 'ema_long_span': 28, 'macd_short_span': 13, 'macd_long_span': 25, 'macd_signal_span': 9},
                {'rsi': (70, 45), 'ema_short_span': 14, 'ema_long_span': 32, 'macd_short_span': 14, 'macd_long_span': 24, 'macd_signal_span': 10},
                {'rsi': (55, 35), 'ema_short_span': 8,  'ema_long_span': 25, 'macd_short_span': 11, 'macd_long_span': 24, 'macd_signal_span': 8},
                {'rsi': (60, 40), 'ema_short_span': 11, 'ema_long_span': 29, 'macd_short_span': 12, 'macd_long_span': 27, 'macd_signal_span': 10},
                {'rsi': (65, 45), 'ema_short_span': 13, 'ema_long_span': 31, 'macd_short_span': 14, 'macd_long_span': 28, 'macd_signal_span': 11},
                {'rsi': (70, 50), 'ema_short_span': 15, 'ema_long_span': 35, 'macd_short_span': 15, 'macd_long_span': 30, 'macd_signal_span': 12},
                {'rsi': (58, 38), 'ema_short_span': 9,  'ema_long_span': 26, 'macd_short_span': 12, 'macd_long_span': 25, 'macd_signal_span': 9},
                {'rsi': (63, 43), 'ema_short_span': 12, 'ema_long_span': 30, 'macd_short_span': 13, 'macd_long_span': 27, 'macd_signal_span': 10},
                {'rsi': (68, 48), 'ema_short_span': 14, 'ema_long_span': 34, 'macd_short_span': 14, 'macd_long_span': 28, 'macd_signal_span': 11},
                # Add more combinations as needed
            ],
            'margin': 0.1
        },
        'USDCAD': {
            'symbol': 'USDCAD',
            'timeframe': mt5.TIMEFRAME_D1,
            'mean_candle_size': 0.0085,
            'buy_prediction_file': 'pred_USDCAD_Buy copy.csv',
            'sell_prediction_file': 'pred_USDCAD_Sell.csv',
            'buy_risk_reward_ratio': "2:2",
            'sell_risk_reward_ratio': "1:1",
            'parameter_combos': [
                {'rsi': (60, 40), 'ema_short_span': 10, 'ema_long_span': 30, 'macd_short_span': 12, 'macd_long_span': 26, 'macd_signal_span': 9},
                {'rsi': (65, 40), 'ema_short_span': 12, 'ema_long_span': 28, 'macd_short_span': 13, 'macd_long_span': 25, 'macd_signal_span': 9},
                {'rsi': (70, 45), 'ema_short_span': 14, 'ema_long_span': 32, 'macd_short_span': 14, 'macd_long_span': 24, 'macd_signal_span': 10},
                {'rsi': (55, 35), 'ema_short_span': 8,  'ema_long_span': 25, 'macd_short_span': 11, 'macd_long_span': 24, 'macd_signal_span': 8},
                {'rsi': (60, 40), 'ema_short_span': 11, 'ema_long_span': 29, 'macd_short_span': 12, 'macd_long_span': 27, 'macd_signal_span': 10},
                {'rsi': (65, 45), 'ema_short_span': 13, 'ema_long_span': 31, 'macd_short_span': 14, 'macd_long_span': 28, 'macd_signal_span': 11},
                {'rsi': (70, 50), 'ema_short_span': 15, 'ema_long_span': 35, 'macd_short_span': 15, 'macd_long_span': 30, 'macd_signal_span': 12},
                {'rsi': (58, 38), 'ema_short_span': 9,  'ema_long_span': 26, 'macd_short_span': 12, 'macd_long_span': 25, 'macd_signal_span': 9},
                {'rsi': (63, 43), 'ema_short_span': 12, 'ema_long_span': 30, 'macd_short_span': 13, 'macd_long_span': 27, 'macd_signal_span': 10},
                {'rsi': (68, 48), 'ema_short_span': 14, 'ema_long_span': 34, 'macd_short_span': 14, 'macd_long_span': 28, 'macd_signal_span': 11},
                # Add more combinations as needed
            ],
            'margin': 0.05
        } 
    }

    print("Starting backtesting script...")
    logger.info("Backtesting script started.")
    init_mt5_connection(config['login'], config['password'], config['server'])
    utc_from = datetime(2024, 1, 1, tzinfo=pytz.utc)
    utc_to = datetime(2025, 1, 9, tzinfo=pytz.utc)

    results_summary = []

    for pair_name, pair_cfg in config.items():
        if pair_name in ['login', 'password', 'server']:
            continue

        print(f"\nProcessing pair: {pair_name}")
        logger.info(f"Processing pair: {pair_name}")

        df_ohlc = fetch_ohlc_data(
            pair_cfg['symbol'], pair_cfg['timeframe'], utc_from, utc_to
        )
        if df_ohlc is None or df_ohlc.empty:
            warning_msg = f"No data for {pair_name}, skipping."
            logger.warning(warning_msg)
            print(f"Warning: {warning_msg}")
            continue

        # Initially add technical indicators with default parameters
        df_ohlc = add_technical_indicators(df_ohlc)
        logger.info(f"Added technical indicators for {pair_name}.")
        print(f"Added technical indicators for {pair_name}.")

        # Optional: Plot RSI & MAs (currently commented out)
        pair_folder = os.path.join("plots", pair_name)
        os.makedirs(pair_folder, exist_ok=True)

        # Uncomment below if you want to plot RSI and MAs
        """
        plt.figure(figsize=(10, 5))
        plt.plot(df_ohlc['time'], df_ohlc['rsi'], label='RSI')
        plt.title(f'RSI for {pair_name}')
        plt.grid(True)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
        plt.legend()
        rsi_plot_file = os.path.join(pair_folder, f"{pair_name}_RSI.png")
        plt.savefig(rsi_plot_file)
        plt.close()

        plt.figure(figsize=(10, 5))
        plt.plot(df_ohlc['time'], df_ohlc['short_ma'], label='Short MA', color='blue')
        plt.plot(df_ohlc['time'], df_ohlc['long_ma'], label='Long MA', color='orange')
        plt.title(f'Moving Averages for {pair_name}')
        plt.grid(True)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
        plt.legend()
        ma_plot_file = os.path.join(pair_folder, f"{pair_name}_MA.png")
        plt.savefig(ma_plot_file)
        plt.close()
        """

        # For each signal type
        for signal_type in ['Buy', 'Sell']:
            pred_file_key = f"{signal_type.lower()}_prediction_file"
            rr_key = f"{signal_type.lower()}_risk_reward_ratio"
            if pred_file_key not in pair_cfg or rr_key not in pair_cfg:
                warning_msg = f"No config for {pair_name} {signal_type}"
                logger.warning(warning_msg)
                print(f"Warning: {warning_msg}. Skipping this signal type.")
                continue

            pred_file = pair_cfg[pred_file_key]
            logger.info(f"Loading prediction file: {pred_file}")
            print(f"  Loading prediction file: {pred_file}")
            df_pred = load_and_align_data(df_ohlc, pred_file)
            if df_pred is None or df_pred.empty:
                warning_msg = f"No predictions for {pair_name} {signal_type}, skipping."
                logger.warning(warning_msg)
                print(f"  Warning: {warning_msg}")
                continue

            risk_reward_ratio = pair_cfg[rr_key]
            mean_candle_size = pair_cfg.get('mean_candle_size', 0.005)
            parameter_combos = pair_cfg.get('parameter_combos', [{
                'rsi': (60, 40),
                'ema_short_span': 10,
                'ema_long_span': 30,
                'macd_short_span': 12,
                'macd_long_span': 26,
                'macd_signal_span': 9
            }])
            margin = pair_cfg['margin']

            # 1) Filter approach (multiple parameter combos)
            run_filter_variations(
                df_pred,
                pair_name,
                signal_type,
                margin,
                risk_reward_ratio,
                mean_candle_size,
                parameter_combos,
                results_summary
            )

            # 2) WithoutFilter approach (once)
            run_without_filter_once(
                df_pred,
                pair_name,
                signal_type,
                margin,
                risk_reward_ratio,
                mean_candle_size,
                results_summary
            )

    # ============================
    # Summarize Results
    # ============================

    summarize_results(results_summary)
    mt5.shutdown()
    logger.info("Disconnected from MetaTrader 5.")
    print("\nBacktesting completed. Disconnected from MetaTrader 5.")

if __name__ == "__main__":
    main()


In [ ]:
import sys
import os
import logging
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pytz

import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
from ta.momentum import RSIIndicator

# Logging configuration
logging.basicConfig(
    filename='backtest.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit("MT5 initialization failed.")
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

def fetch_ohlc_data(symbol, timeframe, start_date, end_date):
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    if data is None or len(data) == 0:
        logging.error(f"Failed to fetch data for {symbol}")
        return None
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.rename(
        columns={
            'open': 'Open',
            'high': 'High',
            'low': 'Low',
            'close': 'Close',
            'tick_volume': 'Volume'
        },
        inplace=True
    )
    return df[['time', 'Open', 'High', 'Low', 'Close', 'Volume']]

def add_technical_indicators(ohlc_data):
    ohlc_data['rsi'] = RSIIndicator(ohlc_data['Close'], window=14).rsi()
    ohlc_data['short_ma'] = ohlc_data['Close'].ewm(span=10, adjust=False).mean()
    ohlc_data['long_ma'] = ohlc_data['Close'].ewm(span=30, adjust=False).mean()
    short_ema = ohlc_data['Close'].ewm(span=12, adjust=False).mean()
    long_ema = ohlc_data['Close'].ewm(span=26, adjust=False).mean()
    ohlc_data['macd'] = short_ema - long_ema
    ohlc_data['macd_signal'] = ohlc_data['macd'].ewm(span=9, adjust=False).mean()
    return ohlc_data

def load_and_align_data(ohlc_data, prediction_file):
    try:
        pred_df = pd.read_csv(prediction_file, parse_dates=['time'])
        if 'prediction' not in pred_df.columns:
            logging.error(f"'prediction' column not found in {prediction_file}")
            return None
    except Exception as e:
        logging.error(f"Error loading prediction file {prediction_file}: {e}")
        return None

    merged = pd.merge(
        ohlc_data,
        pred_df[['time', 'prediction']],
        on='time',
        how='left'
    )
    merged['prediction'] = merged['prediction'].fillna(0)
    return merged

def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, trade_type):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part

    if trade_type == "Buy":
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    else:  # "Sell"
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount

    return sl_price, tp_price

class FilterStrategy(Strategy):
    pair_name = None
    signal_type = None
    rsi_buy = 60
    rsi_sell = 40
    risk_reward_ratio = "1:1"        # e.g., "1:2"
    mean_candle_size = 0.005         # e.g., 0.005
    volume_threshold = 1000

    def init(self):
        pass

    def next(self):
        entry_price = self.data.Close[-1]
        buy_cond = (
            (self.data.rsi[-1] < self.rsi_buy) &
            (self.data.short_ma[-1] > self.data.long_ma[-1]) &
            (self.data.macd[-1] > self.data.macd_signal[-1]) &
            (self.data.Volume[-1] > self.volume_threshold)
        )
        sell_cond = (
            (self.data.rsi[-1] > self.rsi_sell) &
            (self.data.short_ma[-1] < self.data.long_ma[-1]) &
            (self.data.macd[-1] < self.data.macd_signal[-1]) &
            (self.data.Volume[-1] > self.volume_threshold)
        )

        if self.signal_type == 'Buy':
            if self.data.prediction[-2] == 1 and buy_cond:
                sl_price, tp_price = calculate_prices(
                    entry_price, self.risk_reward_ratio, 
                    self.mean_candle_size, "Buy"
                )
                self.buy(sl=sl_price, tp=tp_price)
        else: # 'Sell'
            if self.data.prediction[-1] == 1 and sell_cond:
                sl_price, tp_price = calculate_prices(
                    entry_price, self.risk_reward_ratio, 
                    self.mean_candle_size, "Sell"
                )
                self.sell(sl=sl_price, tp=tp_price)

class WithoutFilterStrategy(Strategy):
    pair_name = None
    signal_type = None
    risk_reward_ratio = "1:1"
    mean_candle_size = 0.005

    def init(self):
        pass

    def next(self):
        entry_price = self.data.Close[-1]
        if self.signal_type == 'Buy':
            if self.data.prediction[-1] == 1:
                sl_price, tp_price = calculate_prices(
                    entry_price, self.risk_reward_ratio, 
                    self.mean_candle_size, "Buy"
                )
                self.buy(sl=sl_price, tp=tp_price)
        else:  # 'Sell'
            if self.data.prediction[-1] == 1:
                sl_price, tp_price = calculate_prices(
                    entry_price, self.risk_reward_ratio, 
                    self.mean_candle_size, "Sell"
                )
                self.sell(sl=sl_price, tp=tp_price)

def run_filter_variations(
    ohlc_data, pair_name, signal_type, margin, risk_reward_ratio, 
    mean_candle_size, rsi_combos, results_summary
):
    for (rsi_buy, rsi_sell) in rsi_combos:
        FilterStrategy.pair_name = pair_name
        FilterStrategy.signal_type = signal_type
        FilterStrategy.rsi_buy = rsi_buy
        FilterStrategy.rsi_sell = rsi_sell
        FilterStrategy.risk_reward_ratio = risk_reward_ratio
        FilterStrategy.mean_candle_size = mean_candle_size

        bt = Backtest(
            ohlc_data.set_index('time'),
            FilterStrategy,
            cash=1000,
            commission=0.0003,
            margin=margin
        )
        stats = bt.run()
        # Access the equity curve
        eq_curve = stats['_equity_curve'].copy()

        pair_folder = f"plots/{pair_name}"
        os.makedirs(pair_folder, exist_ok=True)

        plt.figure(figsize=(10, 6))
        plt.plot(eq_curve.index, eq_curve['Equity'], label=f"{pair_name} {signal_type} Filter")
        plt.title(f"{pair_name} - {signal_type} Filter\nRSI: {rsi_buy}/{rsi_sell}, RR: {risk_reward_ratio}")
        plt.xlabel("Time")
        plt.ylabel("Equity")
        plt.legend()
        plt.grid(True)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
        plt.gcf().autofmt_xdate()  # Rotate date labels
        plot_filename = os.path.join(
            pair_folder,
           f"{pair_name}_{signal_type}_Filter_RSI_{rsi_buy}_{rsi_sell}_RR_{risk_reward_ratio.replace(':','_')}.png")
        plt.savefig(plot_filename)
        plt.show()

        results_summary.append({
            "Pair": pair_name,
            "Signal": signal_type,
            "Approach": "Filter",
            "RSI (Buy/Sell)": f"{rsi_buy}/{rsi_sell}",
            "Risk/Reward": risk_reward_ratio,
            "Final Equity": stats['Equity Final [$]'],
            "Return (%)": stats['Return [%]'],
            "Max Drawdown (%)": stats['Max. Drawdown [%]'],
            "Profit Factor": stats['Profit Factor'],
            "Sharpe Ratio": stats['Sharpe Ratio'],
            "Win Rate (%)": stats['Win Rate [%]'],
            "Total Trades": stats['# Trades'],
            "Avg Trade Duration": stats['Avg. Trade Duration'],
            "Equity Curve Image": plot_filename,
        })

def run_without_filter_once(
    ohlc_data, pair_name, signal_type, margin, 
    risk_reward_ratio, mean_candle_size, results_summary
):
    WithoutFilterStrategy.pair_name = pair_name
    WithoutFilterStrategy.signal_type = signal_type
    WithoutFilterStrategy.risk_reward_ratio = risk_reward_ratio
    WithoutFilterStrategy.mean_candle_size = mean_candle_size

    bt = Backtest(
        ohlc_data.set_index('time'),
        WithoutFilterStrategy,
        cash=1000,
        commission=0.0003,
        margin=margin
    )
    stats = bt.run()
    # Access the equity curve
    eq_curve = stats['_equity_curve'].copy()

    pair_folder = f"plots/{pair_name}"
    os.makedirs(pair_folder, exist_ok=True)

    plt.figure(figsize=(10, 6))
    plt.plot(eq_curve.index, eq_curve['Equity'], label=f"{pair_name} {signal_type} WithoutFilter", color='orange')
    plt.title(f"{pair_name} - {signal_type} WithoutFilter\nRR: {risk_reward_ratio}")
    plt.xlabel("Time")
    plt.ylabel("Equity")
    plt.legend()
    plt.grid(True)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gcf().autofmt_xdate()  # Rotate date labels
    plot_filename = os.path.join(
        pair_folder,
        f"{pair_name}_{signal_type}_WithoutFilter_RR_{risk_reward_ratio.replace(':','_')}.png")
    plt.savefig(plot_filename)
    plt.show()

    results_summary.append({
        "Pair": pair_name,
        "Signal": signal_type,
        "Approach": "WithoutFilter",
        "RSI (Buy/Sell)": "N/A",
        "Risk/Reward": risk_reward_ratio,
        "Final Equity": stats['Equity Final [$]'],
        "Return (%)": stats['Return [%]'],
        "Max Drawdown (%)": stats['Max. Drawdown [%]'],
        "Profit Factor": stats['Profit Factor'],
        "Sharpe Ratio": stats['Sharpe Ratio'],
        "Win Rate (%)": stats['Win Rate [%]'],
        "Total Trades": stats['# Trades'],
        "Avg Trade Duration": stats['Avg. Trade Duration'],
        "Equity Curve Image": plot_filename,
    })

def summarize_results(results_summary):
    if not results_summary:
        print("No results to summarize.")
        return
    df_summary = pd.DataFrame(results_summary)
    print("\nBacktest Summary:\n")
    print(df_summary.to_string(index=False))
    df_summary.to_csv("backtest_results_summary.csv", index=False)
    print("\nResults summary saved to backtest_results_summary.csv")

def main():
    config = {
        'login': 51988090,
        'password': '1fMdV52$74EOcw',
        'server': 'ICMarketsEU-Demo',

        # Each pair has mean_candle_size, and risk-reward ratio in "X:Y" format
        'EURUSD': {
            'symbol': 'EURUSD',
            'timeframe': mt5.TIMEFRAME_D1,
            'mean_candle_size': 0.005,
            'buy_prediction_file': 'pred_EURUSD_Buy.csv',
            'sell_prediction_file': 'pred_AUDUSD_Sell.csv',
            'buy_risk_reward_ratio': "1:1",
            'sell_risk_reward_ratio': "1:1",
            'rsi_combos': [(60, 40),(65, 40),(70, 45),(60, 45),(65, 50),(70, 55),(55, 45),(60, 50),(65, 55),(50, 40)],
            'margin': 0.01
        },
        'GBPUSD': {
            'symbol': 'GBPUSD',
            'timeframe': mt5.TIMEFRAME_D1,
            'mean_candle_size': 0.01,
            'buy_prediction_file': 'pred_GBPUSD_Buy.csv',
            'sell_prediction_file': 'pred_GBPUSD_Sell.csv',
            'buy_risk_reward_ratio': "1:1",
            'sell_risk_reward_ratio': "1:1",
            'rsi_combos': [(60, 40),(65, 40),(70, 45),(60, 45),(65, 50),(70, 55),(55, 45),(60, 50),(65, 55),(50, 40)],
            'margin': 0.1
        },
        'AUDUSD': {
            'symbol': 'AUDUSD',
            'timeframe': mt5.TIMEFRAME_D1,
            'mean_candle_size': 0.008,
            'buy_prediction_file': 'pred_AUDUSD_Buy.csv',
            'sell_prediction_file': 'pred_AUDUSD_Sell.csv',
            'buy_risk_reward_ratio': "1:1",
            'sell_risk_reward_ratio': "1:1",
            'rsi_combos': [(60, 40),(65, 40),(70, 45),(60, 45),(65, 50),(70, 55),(55, 45),(60, 50),(65, 55),(50, 40)],
            'margin': 0.1
        },
        'USDCAD': {
            'symbol': 'USDCAD',
            'timeframe': mt5.TIMEFRAME_D1,
            'mean_candle_size': 0.0085,
            'buy_prediction_file': 'pred_USDCAD_Buy.csv',
            'sell_prediction_file': 'pred_USDCAD_Sell.csv',
            'buy_risk_reward_ratio': "2:2",
            'sell_risk_reward_ratio': "2:3",
            'rsi_combos': [(60, 40),(65, 40),(70, 45),(60, 45),(65, 50),(70, 55),(55, 45),(60, 50),(65, 55),(50, 40)],
            'margin': 0.1
        } 
    }

    init_mt5_connection(config['login'], config['password'], config['server'])
    utc_from = datetime(2024, 1, 1, tzinfo=pytz.utc)
    utc_to = datetime(2025, 1, 9, tzinfo=pytz.utc)

    results_summary = []

    for pair_name, pair_cfg in config.items():
        if pair_name in ['login', 'password', 'server']:
            continue

        df_ohlc = fetch_ohlc_data(
            pair_cfg['symbol'], pair_cfg['timeframe'], utc_from, utc_to
        )
        if df_ohlc is None or df_ohlc.empty:
            logging.warning(f"No data for {pair_name}, skipping.")
            continue

        df_ohlc = add_technical_indicators(df_ohlc)

        # Optional: Plot RSI & MAs
        pair_folder = f"plots/{pair_name}"
        os.makedirs(pair_folder, exist_ok=True)

       #plt.figure(figsize=(10, 5))
       # plt.plot(df_ohlc['time'], df_ohlc['rsi'], label='RSI')
       # plt.title(f'RSI for {pair_name}')
       # plt.grid(True)
        #plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        #plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
        #plt.legend()
        #rsi_plot_file = os.path.join(pair_folder, f"{pair_name}_RSI.png")
        #plt.savefig(rsi_plot_file)
        #plt.show()

        #plt.figure(figsize=(10, 5))
        #plt.plot(df_ohlc['time'], df_ohlc['short_ma'], label='Short MA', color='blue')
        #plt.plot(df_ohlc['time'], df_ohlc['long_ma'], label='Long MA', color='orange')
        #plt.title(f'Moving Averages for {pair_name}')
        #plt.grid(True)
        #plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        #plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
        #plt.legend()
        #ma_plot_file = os.path.join(pair_folder, f"{pair_name}_MA.png")
        #plt.savefig(ma_plot_file)
        #plt.show()

        # For each signal type

        for signal_type in ['Buy', 'Sell']:
            pred_file_key = f"{signal_type.lower()}_prediction_file"
            rr_key = f"{signal_type.lower()}_risk_reward_ratio"
            if pred_file_key not in pair_cfg or rr_key not in pair_cfg:
                logging.warning(f"No config for {pair_name} {signal_type}")
                continue

            pred_file = pair_cfg[pred_file_key]
            df_pred = load_and_align_data(df_ohlc, pred_file)
            if df_pred is None or df_pred.empty:
                logging.warning(f"No predictions for {pair_name} {signal_type}, skipping.")
                continue

            risk_reward_ratio = pair_cfg[rr_key]
            mean_candle_size = pair_cfg.get('mean_candle_size', 0.005)
            rsi_combos = pair_cfg.get('rsi_combos', [(60, 40)])
            margin = pair_cfg['margin']

            # 1) Filter approach (multiple RSI combos)
            run_filter_variations(
                df_pred,
                pair_name,
                signal_type,
                margin,
                risk_reward_ratio,
                mean_candle_size,
                rsi_combos,
                results_summary
            )

            # 2) WithoutFilter approach (once)
            run_without_filter_once(
                df_pred,
                pair_name,
                signal_type,
                margin,
                risk_reward_ratio,
                mean_candle_size,
                results_summary
            )

    summarize_results(results_summary)
    mt5.shutdown()

if __name__ == "__main__":
    main()
